In [3]:
%cd yolov5

/home/ldp/competition/dcic/yolov5


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys
import cv2
from PIL import Image
from IPython.display import display
from utils.general import non_max_suppression, scale_coords
from utils.datasets import exif_transpose, letterbox

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
model = torch.hub.load('/home/ldp/competition/dcic/yolov5-lib-ds', 
                       'custom', 
                       path='/home/ldp/competition/dcic/yolov5/runs/train/32_1024_uflip_vm5_f1_naive2/weights/best.pt',
                       source='local',
                       force_reload=True)# local repo
model.conf = 0.1

In [6]:
submit = pd.read_csv('/home/ldp/competitions/dcic2022/ship_detect/data/submit_example.csv',header=None)

In [7]:
def draw_yolo_predictions(img, bboxes, scores, confthre):
    for i in range(len(bboxes)):
            box = bboxes[i]
            
            score = scores[i]
            if score < confthre:
                continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = int(box[2])
            y1 = int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format('ship', score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

In [13]:
img_size = 256
thres_score = 0.5
for i,test_idx in tqdm(enumerate(submit.loc[:,0])):
    img = cv2.imread(f'/home/ldp/competitions/dcic2022/ship_detect/data/test_dataset/{str(test_idx).zfill(5)}.jpg')
    # display(Image.fromarray(img))
    anno = ''
    scores = []
    boxes = []
    r = model(img, size=img_size*4, augment=True)

    box_count = 0
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        max_score = 0
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > max_score:
                max_score = row.confidence
                max_row = row
            # scores.append(row.confidence)
            # boxes.append([int(row.xmin),int(row.ymin),int(row.xmax),int(row.ymax)])

            if row.confidence >= thres_score:
                obj_width = row.ymax-row.ymin
                obj_length = row.xmax-row.xmin
                if box_count == 0:
                    anno += '{} {} {} {}'.format((row.xmin + obj_length*1/2)/img_size, (row.ymin + obj_width*1/2)/img_size, obj_length/img_size, obj_width/img_size)
                else:
                    anno += ';{} {} {} {}'.format((row.xmin + obj_length*1/2)/img_size, (row.ymin + obj_width*1/2)/img_size, obj_length/img_size, obj_width/img_size)
                box_count += 1

    submit.loc[i,1] = anno



18112it [28:12, 10.70it/s]


In [15]:
submit.to_csv('submission.csv',index=False,header=None)